In [ ]:
!pip install transformers==2.11.0

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device
str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

In [ ]:
import transformers
import os
from transformers import BertTokenizer
from GoEmotions.model import BertForMultiLabelClassification
from GoEmotions.multilabel_pipeline import MultiLabelPipeline

tokenizer = BertTokenizer.from_pretrained("monologg/bert-base-cased-goemotions-original")
model = BertForMultiLabelClassification.from_pretrained("monologg/bert-base-cased-goemotions-original")

goemotions = MultiLabelPipeline(
    model=model,
    tokenizer=tokenizer,
    threshold=0.3,
    device=0
)

In [ ]:
model_dataset = input('Dataset to train model (google, msr, quora, mix, twit0.825 or combined):  ')
training_type = input('Regular or limited training (regular or limited): ')
epochs = int(input('Number of training epochs: '))

testing_set = input('Dataset for prediction generation (google, msr, quora, mix, twit0.825, or combined): ')

if training_type.lower() == 'regular':
    train_set = 'training'
    eval_set = 'testing'
elif training_type.lower() == 'limited':
    train_set = 'testing'
    eval_set = 'training'
else:
    print('Please enter a valid training type')

def print_base_info(model_name, testing_set, eval_emo_filter):
    print(f'---- Scoring Predictions ----')
    print(f'Model: {model_name}')
    print(f'Test Set: {testing_set}-{eval_emo_filter}')
    
emo_filter_list = [
                   't5',
#                   'bart',
#                   'gpt', 
#                   'nil', 
#                   'sid',
#                   'sid_rg', 
                   'emo', 
                   'emo_ge',
#                   'emo_nn',
#                   'emo_sid', 
#                   'emo_sid_nn',
#                   'emo_sid_tg', 
#                   'emo_sid_tg_nn', 
#                   'emo_sid_tg_ge', 
#                   'emo_sid_tg_nn_ge'
                  ]

In [ ]:
high_neg_emo = {'anger', 'disgust', 'grief', 'fear', 'sadness'}
low_neg_emo = {'nervousness', 'annoyance', 'disappointment', 'embarrassment', 'remorse', 'disapproval'}
neutral_emo = {'confusion', 'curiosity', 'realization', 'surprise', 'neutral'}
low_pos_emo = {'approval', 'caring', 'desire', 'relief'}
high_pos_emo = {'amusement', 'excitement', 'pride', 'optimism', 'gratitude', 'joy', 'admiration', 'love'}

high_neg_threshold = -0.4
high_pos_threshold = 0.45

In [ ]:
# Labels the target and predicted texts for scoring by emotion transition and paraphrasing metrics with T5
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()

# Same Sigmoid Function for determining the emotion of a text
threshold = 0.5
def Top_Score_Label (outputs):
    scores = 1 / (1 + np.exp(-outputs))  # Sigmoid
    top_score = 0
    top_label = ""
    for item in scores:
        for idx, s in enumerate(item):
            if s > threshold:
                if s > top_score: 
                    top_label = model.config.id2label[idx]
    return top_label

def labelSid(model_name, eval_emo_filter, df):
    prediction_labels = []
    for index, row in df.iterrows():
        prediction_sid = sid.polarity_scores(rowcalculate_emos_trans.predictions)['compound']

        prediction_labels.append(prediction_sid)
   
    df["prediction_emo"] = prediction_labels

    df.to_csv(f'predictions-data/{model_name}/{testing_set}-{eval_emo_filter}-prediction_emo.tsv', sep='\t')
    

def labelSidRg(model_name, eval_emo_filter, df):
    prediction_labels = []
    for index, row in df.iterrows():
        prediction_sid = sid.polarity_scores(row.predictions)['compound']

        if prediction_sid <= high_neg_threshold:
            prediction_sid_range = 'high_neg'
        elif prediction_sid > high_neg_threshold and prediction_sid < 0:
            prediction_sid_range = 'low_neg'
        elif prediction_sid == 0:
            prediction_sid_range = 'neutral'
        elif prediction_sid < high_pos_threshold and prediction_sid > 0:
            prediction_sid_range = 'low_pos'
        elif prediction_sid >= high_pos_threshold:
            prediction_sid_range = 'high_pos'

        prediction_labels.append(prediction_sid_range)
   
    df["prediction_emo"] = prediction_labels

    df.to_csv(f'predictions-data/{model_name}/{testing_set}-{eval_emo_filter}-prediction_emo.tsv', sep='\t')

    
def labelEmo(model_name, eval_emo_filter, df):
    target_labels = [] 
    prediction_labels = []
    prediction_labels_ge = []
    
    for index, row in df.iterrows():
        t_text = (row.target_text[:512] + '..') if len(row.target_text) > 512 else row.target_text
        target_emo = goemotions(t_text)
        target_label = Top_Score_Label(target_emo)
        target_labels.append(target_label)
        
        p_text = (row.predictions[:512] + '..') if len(row.predictions) > 512 else row.predictions

        prediction_emo = goemotions(p_text)

        prediction_label = Top_Score_Label(prediction_emo)

        prediction_emo_range = 'neutral'
        if prediction_label in high_neg_emo:
            prediction_emo_range = 'high_neg'
        elif prediction_label in low_neg_emo:
            prediction_emo_range = 'low_neg'
        elif prediction_label in neutral_emo:
            prediction_emo_range = 'neutral'
        elif prediction_label in low_pos_emo:
            prediction_emo_range = 'low_pos'
        elif prediction_label in high_pos_emo:
            prediction_emo_range = 'high_pos'

        prediction_labels_ge.append(prediction_label)
        prediction_labels.append(prediction_emo_range)

    df["target_emo_ge"] = target_labels
    df["prediction_emo_ge"] = prediction_labels_ge
    df["prediction_emo"] = prediction_labels

    df.to_csv(f'predictions-data/{model_name}/{testing_set}-{eval_emo_filter}-prediction_emo.tsv', sep='\t')

    
def labelGoEmo(model_name, eval_emo_filter, df):
    target_labels = [] 
    prediction_labels = []
    prediction_labels_ge = []

    for index, row in df.iterrows():
        target_label = row.target_emo
        if target_label in high_neg_emo:
            target_emo_range = 'high_neg'
        elif target_label in low_neg_emo:
            target_emo_range = 'low_neg'
        elif target_label in neutral_emo:
            target_emo_range = 'neutral'
        elif target_label in low_pos_emo:
            target_emo_range = 'low_pos'
        elif target_label in high_pos_emo:
            target_emo_range = 'high_pos'

        target_labels.append(target_emo_range)

        p_text = (row.predictions[:512] + '..') if len(row.predictions) > 512 else row.predictions

        prediction_emo = goemotions(p_text)

        prediction_label = Top_Score_Label(prediction_emo)
        
        prediction_emo_range = 'neutral'
        if prediction_label in high_neg_emo:
            prediction_emo_range = 'high_neg'
        elif prediction_label in low_neg_emo:
            prediction_emo_range = 'low_neg'
        elif prediction_label in neutral_emo:
            prediction_emo_range = 'neutral'
        elif prediction_label in low_pos_emo:
            prediction_emo_range = 'low_pos'
        elif prediction_label in high_pos_emo:
            prediction_emo_range = 'high_pos'

        prediction_labels_ge.append(prediction_label)
        prediction_labels.append(prediction_emo_range)

    df = df.rename(columns={"target_emo": "target_emo_ge"})
    df["target_emo"] = target_labels
    df["prediction_emo_ge"] = prediction_labels_ge
    df["prediction_emo"] = prediction_labels

    df.to_csv(f'predictions-data/{model_name}/{testing_set}-{eval_emo_filter}-prediction_emo.tsv', sep='\t')

    
def labelPredictions(model_emo_filter, eval_emo_filter):
    if (model_emo_filter == 't5'): 
        model_name = 't5-base'
    else:
        model_name = f"{training_type}-{model_dataset}-{model_emo_filter}-{epochs}epochs"
    print(f'---- Labeling Predictions for Model: {model_name} on Dataset: {testing_set}-{eval_emo_filter} ----')

    df = pd.read_csv(f'predictions-data/{model_name}/{testing_set}-{eval_emo_filter}-predictions.tsv', sep='\t').astype(str) 
        
    if model_emo_filter == 'sid':
        labelSid(model_name, eval_emo_filter, df)
    elif model_emo_filter == 'sid_rg':
        labelSidRg(model_name, eval_emo_filter, df)
    elif '_ge' in model_emo_filter or '_ge' in eval_emo_filter or model_emo_filter == 'nil' or eval_emo_filter == 'nil': 
        labelGoEmo(model_name, eval_emo_filter, df)
    else:
        labelEmo(model_name, eval_emo_filter, df)
        
for emo_filter in emo_filter_list:
    if (emo_filter == 't5'): 
#        labelPredictions('t5', 'nil')     
        labelPredictions('t5', 'emo')
        labelPredictions('t5', 'emo_ge')
#        labelPredictions('nil', 'emo_ge')
    else:
        labelPredictions(emo_filter, emo_filter)

In [ ]:
import pandas as pd
import evaluate

from pprint import pprint
from statistics import mean

# Exact Match scores emotion transition
def exact(truths, preds):
    exact = evaluate.load('exact_match')
    result = exact.compute(predictions = preds, references = truths)['exact_match']
    return result

# BLEU, Google_BLEU, ROUGE, and METEOR score paraphrasing
def bleu(truths, preds):
    bleu = evaluate.load('bleu')
    result = bleu.compute(predictions = preds, references = truths)['bleu']
    return result

def google_bleu(truths, preds):
    google_bleu = evaluate.load('google_bleu')
    result = google_bleu.compute(predictions = preds, references = truths)['google_bleu']
    return result

def rouge1(truths, preds):
    rouge = evaluate.load('rouge')
    result = rouge.compute(predictions = preds, references = truths)['rouge1']
    return result
    
def rouge2(truths, preds):
    rouge = evaluate.load('rouge')
    result = rouge.compute(predictions = preds, references = truths)['rouge2']
    return result
    
def rougeL(truths, preds):
    rouge = evaluate.load('rouge')
    result = rouge.compute(predictions = preds, references = truths)['rougeL']
    return result

def bertscore(truths, preds):
    bscore = evaluate.load('bertscore')
    result = bscore.compute(predictions = preds, references = truths, model_type="distilbert-base-uncased")
    return result

def meteor(truths, preds):
    meteor = evaluate.load('meteor')
    result = meteor.compute(predictions = preds, references = truths)['meteor']
    return result


def scoreSid(model_emo_filter, eval_emo_filter):
    if (model_emo_filter == 'T5'): 
        model_name = 't5-base'
    else:
        model_name = f"{training_type}-{model_dataset}-{model_emo_filter}-{epochs}epochs"
    print_base_info(model_name, testing_set, eval_emo_filter)
    df = pd.read_csv(f'predictions-data/{model_name}/{testing_set}-{eval_emo_filter}-prediction_emo.tsv', sep='\t').astype(str)
    
    row_count = 0
    lowered_intensity = 0
    
    for index, row in df.iterrows():
        row_count += 1
        if float(row.input_emo) > float(row.prediction_emo):
            lowered_intensity += 1
    
    pprint("Exact Score")
    pprint(lowered_intensity/row_count)

    pprint("BLEU Score")
    pprint(bleu(df["target_text"], df["predictions"]))

    pprint('Google BLEU Score')
    pprint(google_bleu(df["target_text"], df["predictions"]))

    pprint('ROUGE1 Score')
    pprint(rouge1(df["target_text"], df["predictions"]))

    pprint('ROUGE2 Score')
    pprint(rouge2(df["target_text"], df["predictions"]))

    pprint('ROUGEL Score')
    pprint(rougeL(df["target_text"], df["predictions"]))

    pprint('METEOR Score')
    pprint(meteor(df["target_text"], df["predictions"]))
    
#    pprint('BERTScore')
#    pprint(bertscore(df["target_text"], df["predictions"]))

    print('\n')

def scorePredictions(model_emo_filter, eval_emo_filter):
    if (model_emo_filter == 't5'): 
        model_name = 't5-base'
    else:
        model_name = f"{training_type}-{model_dataset}-{model_emo_filter}-{epochs}epochs"
    print_base_info(model_name, testing_set, eval_emo_filter)
    df = pd.read_csv(f'predictions-data/{model_name}/{testing_set}-{eval_emo_filter}-prediction_emo.tsv', sep='\t').astype(str)
    scores = {'Model Name & Test Set': model_name + ' & ' + f'{testing_set}-{eval_emo_filter}'}
#    scores.update({'Test Set': f'{testing_set}-{eval_emo_filter}'})

#    pprint("Exact Score")
#    pprint(exact(df["target_emo"], df["prediction_emo"]))
    scores.update({'Exact': exact(df["target_emo"], df["prediction_emo"])})

#    if ('target_emo_ge' in df.columns) and ('prediction_emo_ge' in df.columns): 
#        pprint("Exact Score GE")
#        pprint(exact(df["target_emo_ge"], df["prediction_emo_ge"]))
    scores.update({'Exact GE': exact(df["target_emo_ge"], df["prediction_emo_ge"])})

#    pprint("BLEU Score")
#    pprint(bleu(df["target_text"], df["predictions"]))
    scores.update({'BLEU': bleu(df["target_text"], df["predictions"])})

#    pprint('Google BLEU Score')
#    pprint(google_bleu(df["target_text"], df["predictions"]))
    scores.update({'Google BLEU': google_bleu(df["target_text"], df["predictions"])})

#    pprint('ROUGE1 Score')
#    pprint(rouge1(df["target_text"], df["predictions"]))
    scores.update({'ROUGE1': rouge1(df["target_text"], df["predictions"])})

#    pprint('ROUGE2 Score')
#    pprint(rouge2(df["target_text"], df["predictions"]))
    scores.update({'ROUGE2': rouge2(df["target_text"], df["predictions"])})

#    pprint('ROUGEL Score')
#    pprint(rougeL(df["target_text"], df["predictions"]))
    scores.update({'ROUGEL': rougeL(df["target_text"], df["predictions"])})

#    pprint('METEOR Score')
#    pprint(meteor(df["target_text"], df["predictions"]))
    scores.update({'METEOR': meteor(df["target_text"], df["predictions"])})

#    pprint('BERTScore')
#    pprint(bertscore(list(df["target_text"]), list(df["predictions"])))
#    score.update({'BERTScore': bertscore(df["target_text"], df["predictions"])})

    range_trans_counts = {}
    range_trans_hits = {}
    for index, row in df.iterrows():
        if '_ge' in model_emo_filter or '_ge' in eval_emo_filter or model_emo_filter == 'nil' or eval_emo_filter == 'nil':
            if row.input_emo in high_neg_emo:
                input_emo_range = 'high_neg'
            elif row.input_emo in low_neg_emo:
                input_emo_range = 'low_neg'
            elif row.input_emo in neutral_emo:
                input_emo_range = 'neutral'
            elif row.input_emo in low_pos_emo:
                input_emo_range = 'low_pos'
            elif row.input_emo in high_pos_emo:
                input_emo_range = 'high_pos'
            target_emo_range = row.target_emo
        else: 
            input_emo_range = row.input_emo
            target_emo_range = row.target_emo


        key = input_emo_range + " to " + target_emo_range
        if key in range_trans_counts.keys(): 
            range_trans_counts.update({key: range_trans_counts[key]+1})
            if target_emo_range == row.prediction_emo: 
                if key in range_trans_hits.keys():
                    range_trans_hits.update({key: range_trans_hits[key]+1}) 
                else:
                    range_trans_hits.update({key: 1}) 
        else:
            range_trans_counts.update({key: 1})
            if target_emo_range == row.prediction_emo: 
                if key in range_trans_hits.keys():
                    range_trans_hits.update({key: range_trans_hits[key]+1}) 
                else:
                    range_trans_hits.update({key: 1}) 

                
    for key, value in range_trans_counts.items(): 
        if (key in range_trans_hits): 
            hits = range_trans_hits[key]
        else: 
            hits = 0 
#        print(key + ': ', str(hits/value))
        scores.update({key: hits/value})

    print('\n')
            
    for key, value in scores.items(): 
        print(key + ': ', str(value))
    print('\n')

    return scores

scores_df = pd.DataFrame()

for emo_filter in emo_filter_list:
    if emo_filter == 'sid':
        scoreSid(emo_filter, emo_filter)
    else:
        if (emo_filter == 't5'): 
#            scores = scorePredictions('t5', 'nil')
#            scores_df = scores_df.append(scores, ignore_index=True)
            scores = scorePredictions('t5', 'emo')
            scores_df = scores_df.append(scores, ignore_index=True)
            scores = scorePredictions('t5', 'emo_ge')
            scores_df = scores_df.append(scores, ignore_index=True)
#            scores = scorePredictions('nil', 'emo_ge')
#            scores_df = scores_df.append(scores, ignore_index=True)
        else: 
            scores = scorePredictions(emo_filter, emo_filter)
            scores_df = scores_df.append(scores, ignore_index=True)

print(scores_df)
scores_df.to_csv(f'predictions-data/{model_dataset}-{training_type}-{epochs}epochs-all-scores.tsv', sep='\t')

In [ ]:
labelPredictions('emo', 'emo_sid_tg_nn')
scorePredictions('emo', 'emo_sid_tg_nn')

labelPredictions('emo_ge', 'emo_sid_tg_nn_ge')
scorePredictions('emo_ge', 'emo_sid_tg_nn_ge')

In [ ]:
labelPredictions('emo', 'emo') 
scorePredictions('emo', 'emo') 

labelPredictions('t5', 'emo') 
scorePredictions('t5', 'emo') 

labelPredictions('emo_ge', 'emo_ge') 
scorePredictions('emo_ge', 'emo_ge') 

labelPredictions('t5', 'emo_ge') 
scorePredictions('t5', 'emo_ge') 

In [ ]:
labelPredictions('t5', 'emo_sid_tg_nn_ge') 
scorePredictions('t5', 'emo_sid_tg_nn_ge') 

In [ ]:
labelPredictions('emo', 'emo_sid_tg_nn') 
labelPredictions('emo_nn', 'emo_sid_tg_nn') 
labelPredictions('emo_sid', 'emo_sid_tg_nn') 
labelPredictions('emo_sid_nn', 'emo_sid_tg_nn') 
labelPredictions('emo_sid_tg', 'emo_sid_tg_nn') 
labelPredictions('emo_sid_tg_nn', 'emo_sid_tg_nn') 
labelPredictions('emo_sid_tg_ge', 'emo_sid_tg_nn_ge') 
labelPredictions('emo_sid_tg_nn_ge', 'emo_sid_tg_nn_ge') 

scorePredictions('emo', 'emo_sid_tg_nn') 
scorePredictions('emo_nn', 'emo_sid_tg_nn') 
scorePredictions('emo_sid', 'emo_sid_tg_nn') 
scorePredictions('emo_sid_nn', 'emo_sid_tg_nn') 
scorePredictions('emo_sid_tg', 'emo_sid_tg_nn') 
scorePredictions('emo_sid_tg_nn', 'emo_sid_tg_nn') 
scorePredictions('emo_sid_tg_ge', 'emo_sid_tg_nn_ge') 
scorePredictions('emo_sid_tg_nn_ge', 'emo_sid_tg_nn_ge') 

In [ ]:
# Labels the target and predicted texts for scoring by emotion transition and paraphrasing metrics with Bart
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()

# Same Sigmoid Function for determining the emotion of a text
threshold = 0.5
def Top_Score_Label (outputs):
    scores = 1 / (1 + np.exp(-outputs))  # Sigmoid
    top_score = 0
    top_label = ""
    for item in scores:
        for idx, s in enumerate(item):
            if s > threshold:
                if s > top_score: 
                    top_label = model.config.id2label[idx]
    return top_label

def labelSid(model_name, eval_emo_filter, df):
    prediction_labels = []
    for index, row in df.iterrows():
        prediction_sid = sid.polarity_scores(rowcalculate_emos_trans.predictions)['compound']

        prediction_labels.append(prediction_sid)
   
    df["prediction_emo"] = prediction_labels

    df.to_csv(f'predictions-data/{model_name}/{testing_set}-{eval_emo_filter}-prediction_emo.tsv', sep='\t')
    

def labelSidRg(model_name, eval_emo_filter, df):
    prediction_labels = []
    for index, row in df.iterrows():
        prediction_sid = sid.polarity_scores(row.predictions)['compound']

        if prediction_sid <= high_neg_threshold:
            prediction_sid_range = 'high_neg'
        elif prediction_sid > high_neg_threshold and prediction_sid < 0:
            prediction_sid_range = 'low_neg'
        elif prediction_sid == 0:
            prediction_sid_range = 'neutral'
        elif prediction_sid < high_pos_threshold and prediction_sid > 0:
            prediction_sid_range = 'low_pos'
        elif prediction_sid >= high_pos_threshold:
            prediction_sid_range = 'high_pos'

        prediction_labels.append(prediction_sid_range)
   
    df["prediction_emo"] = prediction_labels

    df.to_csv(f'predictions-data/{model_name}/{testing_set}-{eval_emo_filter}-prediction_emo.tsv', sep='\t')

    
def labelEmo(model_name, eval_emo_filter, df):
    target_labels = [] 
    prediction_labels = []
    prediction_labels_ge = []
    
    for index, row in df.iterrows():
        t_text = (row.target_text[:512] + '..') if len(row.target_text) > 512 else row.target_text
        target_emo = goemotions(t_text)
        target_label = Top_Score_Label(target_emo)
        target_labels.append(target_label)
        
        p_text = (row.predictions[:512] + '..') if len(row.predictions) > 512 else row.predictions

        prediction_emo = goemotions(p_text)

        prediction_label = Top_Score_Label(prediction_emo)

        prediction_emo_range = 'neutral'
        if prediction_label in high_neg_emo:
            prediction_emo_range = 'high_neg'
        elif prediction_label in low_neg_emo:
            prediction_emo_range = 'low_neg'
        elif prediction_label in neutral_emo:
            prediction_emo_range = 'neutral'
        elif prediction_label in low_pos_emo:
            prediction_emo_range = 'low_pos'
        elif prediction_label in high_pos_emo:
            prediction_emo_range = 'high_pos'

        prediction_labels_ge.append(prediction_label)
        prediction_labels.append(prediction_emo_range)

    df["target_emo_ge"] = target_labels
    df["prediction_emo_ge"] = prediction_labels_ge
    df["prediction_emo"] = prediction_labels

    df.to_csv(f'predictions-data/{model_name}/{testing_set}-{eval_emo_filter}-prediction_emo.tsv', sep='\t')

    
def labelGoEmo(model_name, eval_emo_filter, df):
    target_labels = [] 
    prediction_labels = []
    prediction_labels_ge = []

    for index, row in df.iterrows():
        target_label = row.target_emo
        if target_label in high_neg_emo:
            target_emo_range = 'high_neg'
        elif target_label in low_neg_emo:
            target_emo_range = 'low_neg'
        elif target_label in neutral_emo:
            target_emo_range = 'neutral'
        elif target_label in low_pos_emo:
            target_emo_range = 'low_pos'
        elif target_label in high_pos_emo:
            target_emo_range = 'high_pos'

        target_labels.append(target_emo_range)

        p_text = (row.predictions[:512] + '..') if len(row.predictions) > 512 else row.predictions

        prediction_emo = goemotions(p_text)

        prediction_label = Top_Score_Label(prediction_emo)
        
        if prediction_label in high_neg_emo:
            prediction_emo_range = 'high_neg'
        elif prediction_label in low_neg_emo:
            prediction_emo_range = 'low_neg'
        elif prediction_label in neutral_emo:
            prediction_emo_range = 'neutral'
        elif prediction_label in low_pos_emo:
            prediction_emo_range = 'low_pos'
        elif prediction_label in high_pos_emo:
            prediction_emo_range = 'high_pos'

        prediction_labels_ge.append(prediction_label)
        prediction_labels.append(prediction_emo_range)

    df = df.rename(columns={"target_emo": "target_emo_ge"})
    df["target_emo"] = target_labels
    df["prediction_emo_ge"] = prediction_labels_ge
    df["prediction_emo"] = prediction_labels

    df.to_csv(f'predictions-data/{model_name}/{testing_set}-{eval_emo_filter}-prediction_emo.tsv', sep='\t')

    
def labelPredictions(model_emo_filter, eval_emo_filter):
    if (model_emo_filter == 'bart'): 
        model_name = 'facebook/bart-base'       
    else:
        model_name = f"{training_type}-{model_dataset}-{model_emo_filter}-{epochs}epochs-bart"
    print(f'---- Labeling Predictions for Model: {model_name} on Dataset: {testing_set}-{eval_emo_filter} ----')

    df = pd.read_csv(f'predictions-data/{model_name}/{testing_set}-{eval_emo_filter}-predictions.tsv', sep='\t').astype(str)       
        
    if model_emo_filter == 'sid':
        labelSid(model_name, eval_emo_filter, df)
    elif model_emo_filter == 'sid_rg':
        labelSidRg(model_name, eval_emo_filter, df)
    elif '_ge' in model_emo_filter or '_ge' in eval_emo_filter or model_emo_filter == 'nil' or eval_emo_filter == 'nil': 
        labelGoEmo(model_name, eval_emo_filter, df)
    else:
        labelEmo(model_name, eval_emo_filter, df)
        
for emo_filter in emo_filter_list:
    if (emo_filter == 'bart'):
        labelPredictions('bart', 'emo')
        labelPredictions('bart', 'emo_ge')     
    else:
        labelPredictions(emo_filter, emo_filter)

In [ ]:
import pandas as pd
import evaluate

from pprint import pprint
from statistics import mean

# Exact Match scores emotion transition
def exact(truths, preds):
    exact = evaluate.load('exact_match')
    result = exact.compute(predictions = preds, references = truths)['exact_match']
    return result

# BLEU, Google_BLEU, ROUGE, and METEOR score paraphrasing
def bleu(truths, preds):
    bleu = evaluate.load('bleu')
    result = bleu.compute(predictions = preds, references = truths)['bleu']
    return result

def google_bleu(truths, preds):
    google_bleu = evaluate.load('google_bleu')
    result = google_bleu.compute(predictions = preds, references = truths)['google_bleu']
    return result

def rouge1(truths, preds):
    rouge = evaluate.load('rouge')
    result = rouge.compute(predictions = preds, references = truths)['rouge1']
    return result
    
def rouge2(truths, preds):
    rouge = evaluate.load('rouge')
    result = rouge.compute(predictions = preds, references = truths)['rouge2']
    return result
    
def rougeL(truths, preds):
    rouge = evaluate.load('rouge')
    result = rouge.compute(predictions = preds, references = truths)['rougeL']
    return result

def bertscore(truths, preds):
    bscore = evaluate.load('bertscore')
    result = bscore.compute(predictions = preds, references = truths, model_type="distilbert-base-uncased")
    return result

def meteor(truths, preds):
    meteor = evaluate.load('meteor')
    result = meteor.compute(predictions = preds, references = truths)['meteor']
    return result


def scoreSid(model_emo_filter, eval_emo_filter):
    if (model_emo_filter == 'T5'): 
        model_name = 't5-base'
    else:
        model_name = f"{training_type}-{model_dataset}-{model_emo_filter}-{epochs}epochs"
    print_base_info(model_name, testing_set, eval_emo_filter)
    df = pd.read_csv(f'predictions-data/{model_name}/{testing_set}-{eval_emo_filter}-prediction_emo.tsv', sep='\t').astype(str)
    
    row_count = 0
    lowered_intensity = 0
    
    for index, row in df.iterrows():
        row_count += 1
        if float(row.input_emo) > float(row.prediction_emo):
            lowered_intensity += 1
    
    pprint("Exact Score")
    pprint(lowered_intensity/row_count)

    pprint("BLEU Score")
    pprint(bleu(df["target_text"], df["predictions"]))

    pprint('Google BLEU Score')
    pprint(google_bleu(df["target_text"], df["predictions"]))

    pprint('ROUGE1 Score')
    pprint(rouge1(df["target_text"], df["predictions"]))

    pprint('ROUGE2 Score')
    pprint(rouge2(df["target_text"], df["predictions"]))

    pprint('ROUGEL Score')
    pprint(rougeL(df["target_text"], df["predictions"]))

    pprint('METEOR Score')
    pprint(meteor(df["target_text"], df["predictions"]))

    print('\n')

def scorePredictions(model_emo_filter, eval_emo_filter):
    if (model_emo_filter == 'bart'): 
        model_name = 'facebook/bart-base'
    else:
        model_name = f"{training_type}-{model_dataset}-{model_emo_filter}-{epochs}epochs"
    print_base_info(model_name, testing_set, eval_emo_filter)
    df = pd.read_csv(f'predictions-data/{model_name}/{testing_set}-{eval_emo_filter}-prediction_emo.tsv', sep='\t').astype(str)

    scores = {'Model Name & Test Set': model_name + ' & ' + f'{testing_set}-{eval_emo_filter}'}
#    scores.update({'Test Set': f'{testing_set}-{eval_emo_filter}'})

#    pprint("Exact Score")
#    pprint(exact(df["target_emo"], df["prediction_emo"]))
    scores.update({'Exact': exact(df["target_emo"], df["prediction_emo"])})

#    if ('target_emo_ge' in df.columns) and ('prediction_emo_ge' in df.columns): 
#        pprint("Exact Score GE")
#        pprint(exact(df["target_emo_ge"], df["prediction_emo_ge"]))
    scores.update({'Exact GE': exact(df["target_emo_ge"], df["prediction_emo_ge"])})

#    pprint("BLEU Score")
#    pprint(bleu(df["target_text"], df["predictions"]))
    scores.update({'BLEU': bleu(df["target_text"], df["predictions"])})

#    pprint('Google BLEU Score')
#    pprint(google_bleu(df["target_text"], df["predictions"]))
    scores.update({'Google BLEU': google_bleu(df["target_text"], df["predictions"])})

#    pprint('ROUGE1 Score')
#    pprint(rouge1(df["target_text"], df["predictions"]))
    scores.update({'ROUGE1': rouge1(df["target_text"], df["predictions"])})

#    pprint('ROUGE2 Score')
#    pprint(rouge2(df["target_text"], df["predictions"]))
    scores.update({'ROUGE2': rouge2(df["target_text"], df["predictions"])})

#    pprint('ROUGEL Score')
#    pprint(rougeL(df["target_text"], df["predictions"]))
    scores.update({'ROUGEL': rougeL(df["target_text"], df["predictions"])})

#    pprint('METEOR Score')
#    pprint(meteor(df["target_text"], df["predictions"]))
    scores.update({'METEOR': meteor(df["target_text"], df["predictions"])})

    for key, value in scores.items(): 
        print(key + ': ', str(value))
    print('\n')

    return scores

scores_df = pd.DataFrame()

for emo_filter in emo_filter_list:
    if emo_filter == 'sid':
        scoreSid(emo_filter, emo_filter)
    else:
        if (emo_filter == 'bart'):
            scores = scorePredictions('bart', 'emo')
            scores_df = scores_df.append(scores, ignore_index=True)
            scores = scorePredictions('bart', 'emo_ge')
            scores_df = scores_df.append(scores, ignore_index=True)
        else: 
            scores = scorePredictions(emo_filter, emo_filter)
            scores_df = scores_df.append(scores, ignore_index=True)

print(scores_df)
scores_df.to_csv(f'predictions-data/{model_dataset}-{training_type}-{epochs}epochs-all-scores.tsv', sep='\t')

In [ ]:
# Labels the target and predicted texts for scoring by emotion transition and paraphrasing metrics with GPT2
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()

# Same Sigmoid Function for determining the emotion of a text
threshold = 0.5
def Top_Score_Label (outputs):
    scores = 1 / (1 + np.exp(-outputs))  # Sigmoid
    top_score = 0
    top_label = ""
    for item in scores:
        for idx, s in enumerate(item):
            if s > threshold:
                if s > top_score: 
                    top_label = model.config.id2label[idx]
    return top_label

def labelSid(model_name, eval_emo_filter, df):
    prediction_labels = []
    for index, row in df.iterrows():
        prediction_sid = sid.polarity_scores(rowcalculate_emos_trans.predictions)['compound']

        prediction_labels.append(prediction_sid)
   
    df["prediction_emo"] = prediction_labels

    df.to_csv(f'predictions-data/{model_name}/{testing_set}-{eval_emo_filter}-prediction_emo.tsv', sep='\t')
    

def labelSidRg(model_name, eval_emo_filter, df):
    prediction_labels = []
    for index, row in df.iterrows():
        prediction_sid = sid.polarity_scores(row.predictions)['compound']

        if prediction_sid <= high_neg_threshold:
            prediction_sid_range = 'high_neg'
        elif prediction_sid > high_neg_threshold and prediction_sid < 0:
            prediction_sid_range = 'low_neg'
        elif prediction_sid == 0:
            prediction_sid_range = 'neutral'
        elif prediction_sid < high_pos_threshold and prediction_sid > 0:
            prediction_sid_range = 'low_pos'
        elif prediction_sid >= high_pos_threshold:
            prediction_sid_range = 'high_pos'

        prediction_labels.append(prediction_sid_range)
   
    df["prediction_emo"] = prediction_labels

    df.to_csv(f'predictions-data/{model_name}/{testing_set}-{eval_emo_filter}-prediction_emo.tsv', sep='\t')

    
def labelEmo(model_name, eval_emo_filter, df):
    target_labels = [] 
    prediction_labels = []
    prediction_labels_ge = []
    
    for index, row in df.iterrows():
        t_text = (row.target_text[:512] + '..') if len(row.target_text) > 512 else row.target_text
        target_emo = goemotions(t_text)
        target_label = Top_Score_Label(target_emo)
        target_labels.append(target_label)
        
        p_text = (row.predictions[:512] + '..') if len(row.predictions) > 512 else row.predictions

        prediction_emo = goemotions(p_text)

        prediction_label = Top_Score_Label(prediction_emo)

        prediction_emo_range = 'neutral'
        if prediction_label in high_neg_emo:
            prediction_emo_range = 'high_neg'
        elif prediction_label in low_neg_emo:
            prediction_emo_range = 'low_neg'
        elif prediction_label in neutral_emo:
            prediction_emo_range = 'neutral'
        elif prediction_label in low_pos_emo:
            prediction_emo_range = 'low_pos'
        elif prediction_label in high_pos_emo:
            prediction_emo_range = 'high_pos'

        prediction_labels_ge.append(prediction_label)
        prediction_labels.append(prediction_emo_range)

    df["target_emo_ge"] = target_labels
    df["prediction_emo_ge"] = prediction_labels_ge
    df["prediction_emo"] = prediction_labels

    df.to_csv(f'predictions-data/{model_name}/{testing_set}-{eval_emo_filter}-prediction_emo.tsv', sep='\t')

    
def labelGoEmo(model_name, eval_emo_filter, df):
    target_labels = [] 
    prediction_labels = []
    prediction_labels_ge = []

    for index, row in df.iterrows():
        target_label = row.target_emo
        if target_label in high_neg_emo:
            target_emo_range = 'high_neg'
        elif target_label in low_neg_emo:
            target_emo_range = 'low_neg'
        elif target_label in neutral_emo:
            target_emo_range = 'neutral'
        elif target_label in low_pos_emo:
            target_emo_range = 'low_pos'
        elif target_label in high_pos_emo:
            target_emo_range = 'high_pos'

        target_labels.append(target_emo_range)

        p_text = (row.predictions[:512] + '..') if len(row.predictions) > 512 else row.predictions

        prediction_emo = goemotions(p_text)

        prediction_label = Top_Score_Label(prediction_emo)
        
        prediction_emo_range = 'neutral'
        if prediction_label in high_neg_emo:
            prediction_emo_range = 'high_neg'
        elif prediction_label in low_neg_emo:
            prediction_emo_range = 'low_neg'
        elif prediction_label in neutral_emo:
            prediction_emo_range = 'neutral'
        elif prediction_label in low_pos_emo:
            prediction_emo_range = 'low_pos'
        elif prediction_label in high_pos_emo:
            prediction_emo_range = 'high_pos'

        prediction_labels_ge.append(prediction_label)
        prediction_labels.append(prediction_emo_range)

    df = df.rename(columns={"target_emo": "target_emo_ge"})
    df["target_emo"] = target_labels
    df["prediction_emo_ge"] = prediction_labels_ge
    df["prediction_emo"] = prediction_labels

    df.to_csv(f'predictions-data/{model_name}/{testing_set}-{eval_emo_filter}-prediction_emo.tsv', sep='\t')

    
def labelPredictions(model_emo_filter, eval_emo_filter):
    if (model_emo_filter == 'gpt'): 
        model_name = 'gpt'       
    else:
        model_name = f"{training_type}-{model_dataset}-{model_emo_filter}-{epochs}epochs-gpt"
    print(f'---- Labeling Predictions for Model: {model_name} on Dataset: {testing_set}-{eval_emo_filter} ----')

    df = pd.read_csv(f'predictions-data/{model_name}/{testing_set}-{eval_emo_filter}-predictions.tsv', sep='\t').astype(str)       
        
    if model_emo_filter == 'sid':
        labelSid(model_name, eval_emo_filter, df)
    elif model_emo_filter == 'sid_rg':
        labelSidRg(model_name, eval_emo_filter, df)
    elif '_ge' in model_emo_filter or '_ge' in eval_emo_filter or model_emo_filter == 'nil' or eval_emo_filter == 'nil': 
        labelGoEmo(model_name, eval_emo_filter, df)
    else:
        labelEmo(model_name, eval_emo_filter, df)
        
for emo_filter in emo_filter_list:
    if (emo_filter == 'gpt'):
        labelPredictions('gpt', 'emo')
        labelPredictions('gpt', 'emo_ge')     
    else:
        labelPredictions(emo_filter, emo_filter)

In [ ]:
import pandas as pd
import evaluate

from pprint import pprint
from statistics import mean

# Exact Match scores emotion transition
def exact(truths, preds):
    exact = evaluate.load('exact_match')
    result = exact.compute(predictions = preds, references = truths)['exact_match']
    return result

# BLEU, Google_BLEU, ROUGE, and METEOR score paraphrasing
def bleu(truths, preds):
    bleu = evaluate.load('bleu')
    result = bleu.compute(predictions = preds, references = truths)['bleu']
    return result

def google_bleu(truths, preds):
    google_bleu = evaluate.load('google_bleu')
    result = google_bleu.compute(predictions = preds, references = truths)['google_bleu']
    return result

def rouge1(truths, preds):
    rouge = evaluate.load('rouge')
    result = rouge.compute(predictions = preds, references = truths)['rouge1']
    return result
    
def rouge2(truths, preds):
    rouge = evaluate.load('rouge')
    result = rouge.compute(predictions = preds, references = truths)['rouge2']
    return result
    
def rougeL(truths, preds):
    rouge = evaluate.load('rouge')
    result = rouge.compute(predictions = preds, references = truths)['rougeL']
    return result

def bertscore(truths, preds):
    bscore = evaluate.load('bertscore')
    result = bscore.compute(predictions = preds, references = truths, model_type="distilbert-base-uncased")
    return result

def meteor(truths, preds):
    meteor = evaluate.load('meteor')
    result = meteor.compute(predictions = preds, references = truths)['meteor']
    return result


def scoreSid(model_emo_filter, eval_emo_filter):
    if (model_emo_filter == 'T5'): 
        model_name = 't5-base'
    else:
        model_name = f"{training_type}-{model_dataset}-{model_emo_filter}-{epochs}epochs"
    print_base_info(model_name, testing_set, eval_emo_filter)
    df = pd.read_csv(f'predictions-data/{model_name}/{testing_set}-{eval_emo_filter}-prediction_emo.tsv', sep='\t').astype(str)
    
    row_count = 0
    lowered_intensity = 0
    
    for index, row in df.iterrows():
        row_count += 1
        if float(row.input_emo) > float(row.prediction_emo):
            lowered_intensity += 1
    
    pprint("Exact Score")
    pprint(lowered_intensity/row_count)

    pprint("BLEU Score")
    pprint(bleu(df["target_text"], df["predictions"]))

    pprint('Google BLEU Score')
    pprint(google_bleu(df["target_text"], df["predictions"]))

    pprint('ROUGE1 Score')
    pprint(rouge1(df["target_text"], df["predictions"]))

    pprint('ROUGE2 Score')
    pprint(rouge2(df["target_text"], df["predictions"]))

    pprint('ROUGEL Score')
    pprint(rougeL(df["target_text"], df["predictions"]))

    pprint('METEOR Score')
    pprint(meteor(df["target_text"], df["predictions"]))

    print('\n')

def scorePredictions(model_emo_filter, eval_emo_filter):
    if (model_emo_filter == 'gpt'): 
        model_name = 'gpt'
    else:
        model_name = f"{training_type}-{model_dataset}-{model_emo_filter}-{epochs}epochs-gpt"
    print_base_info(model_name, testing_set, eval_emo_filter)
    df = pd.read_csv(f'predictions-data/{model_name}/{testing_set}-{eval_emo_filter}-prediction_emo.tsv', sep='\t').astype(str)

    scores = {'Model Name & Test Set': model_name + ' & ' + f'{testing_set}-{eval_emo_filter}'}
#    scores.update({'Test Set': f'{testing_set}-{eval_emo_filter}'})

#    pprint("Exact Score")
#    pprint(exact(df["target_emo"], df["prediction_emo"]))
    scores.update({'Exact': exact(df["target_emo"], df["prediction_emo"])})

#    if ('target_emo_ge' in df.columns) and ('prediction_emo_ge' in df.columns): 
#        pprint("Exact Score GE")
#        pprint(exact(df["target_emo_ge"], df["prediction_emo_ge"]))
    scores.update({'Exact GE': exact(df["target_emo_ge"], df["prediction_emo_ge"])})

#    pprint("BLEU Score")
#    pprint(bleu(df["target_text"], df["predictions"]))
    scores.update({'BLEU': bleu(df["target_text"], df["predictions"])})

#    pprint('Google BLEU Score')
#    pprint(google_bleu(df["target_text"], df["predictions"]))
    scores.update({'Google BLEU': google_bleu(df["target_text"], df["predictions"])})

#    pprint('ROUGE1 Score')
#    pprint(rouge1(df["target_text"], df["predictions"]))
    scores.update({'ROUGE1': rouge1(df["target_text"], df["predictions"])})

#    pprint('ROUGE2 Score')
#    pprint(rouge2(df["target_text"], df["predictions"]))
    scores.update({'ROUGE2': rouge2(df["target_text"], df["predictions"])})

#    pprint('ROUGEL Score')
#    pprint(rougeL(df["target_text"], df["predictions"]))
    scores.update({'ROUGEL': rougeL(df["target_text"], df["predictions"])})

#    pprint('METEOR Score')
#    pprint(meteor(df["target_text"], df["predictions"]))
    scores.update({'METEOR': meteor(df["target_text"], df["predictions"])})

    for key, value in scores.items(): 
        print(key + ': ', str(value))
    print('\n')

    return scores

scores_df = pd.DataFrame()

for emo_filter in emo_filter_list:
    if emo_filter == 'sid':
        scoreSid(emo_filter, emo_filter)
    else:
        if (emo_filter == 'gpt'):
            scores = scorePredictions('gpt', 'emo')
            scores_df = scores_df.append(scores, ignore_index=True)
            scores = scorePredictions('gpt', 'emo_ge')
            scores_df = scores_df.append(scores, ignore_index=True)
        else: 
            scores = scorePredictions(emo_filter, emo_filter)
            scores_df = scores_df.append(scores, ignore_index=True)

print(scores_df)
scores_df.to_csv(f'predictions-data/{model_dataset}-{training_type}-{epochs}epochs-all-scores.tsv', sep='\t')

In [ ]:
# Below are the old GPT2 label and score procedures 

# Labels the target and predicted texts for scoring by emotion transition and paraphrasing metrics with GPT2
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()

# Same Sigmoid Function for determining the emotion of a text
threshold = 0.5
def Top_Score_Label (outputs):
    scores = 1 / (1 + np.exp(-outputs))  # Sigmoid
    top_score = 0
    top_label = ""
    for item in scores:
        for idx, s in enumerate(item):
            if s > threshold:
                if s > top_score: 
                    top_label = model.config.id2label[idx]
    return top_label

def labelSidGPT(model_name, eval_emo_filter, df):
    prediction_labels = []
    for index, row in df.iterrows():
        prediction_sid = sid.polarity_scores(row.predictions)['compound']

        prediction_labels.append(prediction_sid)
   
    df["prediction_emo"] = prediction_labels

    df.to_csv(f'predictions-data/{model_name}/{testing_set}-{eval_emo_filter}-prediction_emo.tsv', sep='\t')
    

def labelSidRgGPT(model_name, eval_emo_filter, df):
    prediction_labels = []
    for index, row in df.iterrows():
        prediction_sid = sid.polarity_scores(row.predictions)['compound']

        if prediction_sid <= high_neg_threshold:
            prediction_sid_range = 'high_neg'
        elif prediction_sid > high_neg_threshold and prediction_sid < 0:
            prediction_sid_range = 'low_neg'
        elif prediction_sid == 0:
            prediction_sid_range = 'neutral'
        elif prediction_sid < high_pos_threshold and prediction_sid > 0:
            prediction_sid_range = 'low_pos'
        elif prediction_sid >= high_pos_threshold:
            prediction_sid_range = 'high_pos'

        prediction_labels.append(prediction_sid_range)
   
    df["prediction_emo"] = prediction_labels

    df.to_csv(f'predictions-data/{model_name}/{testing_set}-{eval_emo_filter}-prediction_emo.tsv', sep='\t')

    
def labelEmoGPT(model_name, eval_emo_filter, df):
    prediction_labels = []
    for index, row in df.iterrows():
        p_text = (row.predictions[:512] + '..') if len(row.predictions) > 512 else row.predictions

        prediction_emo = goemotions(p_text)

        prediction_label = Top_Score_Label(prediction_emo)

        if prediction_label in high_neg_emo:
            prediction_emo_range = 'high_neg'
        elif prediction_label in low_neg_emo:
            prediction_emo_range = 'low_neg'
        elif prediction_label in neutral_emo:
            prediction_emo_range = 'neutral'
        elif prediction_label in low_pos_emo:
            prediction_emo_range = 'low_pos'
        elif prediction_label in high_pos_emo:
            prediction_emo_range = 'high_pos'

        prediction_labels.append(prediction_emo_range)
   
    df["prediction_emo"] = prediction_labels

    df.to_csv(f'predictions-data/{model_name}/{testing_set}-{eval_emo_filter}-prediction_emo.tsv', sep='\t')

    
def labelGoEmoGPT(model_name, eval_emo_filter, df):
    prediction_labels = []
    for index, row in df.iterrows():
        p_text = (row.predictions[:512] + '..') if len(row.predictions) > 512 else row.predictions

        prediction_emo = goemotions(p_text)

        prediction_label = Top_Score_Label(prediction_emo)
        
        prediction_labels.append(prediction_label)
   
    df["prediction_emo"] = prediction_labels

    df.to_csv(f'predictions-data/{model_name}/{testing_set}-{eval_emo_filter}-prediction_emo.tsv', sep='\t')

    
def labelPredictionsGPT(model_emo_filter, eval_emo_filter):
    model_name = f"{training_type}-{model_dataset}-{model_emo_filter}-{epochs}epochs-gpt"
    print(f'---- Labeling Predictions for Model: {model_name} on Dataset: {testing_set}-{eval_emo_filter} ----')

    df = pd.read_csv(f'predictions-data/{model_name}/{testing_set}-{eval_emo_filter}-predictions.tsv', sep='\t').astype(str) 
    
    if model_emo_filter == 'sid':
        labelSidGPT(model_name, eval_emo_filter, df)
    elif model_emo_filter == 'sid_rg':
        labelSidRgGPT(model_name, eval_emo_filter, df)
    elif '_ge' in model_emo_filter: 
        labelGoEmoGPT(model_name, eval_emo_filter, df)
    else:
        labelEmoGPT(model_name, eval_emo_filter, df)
        
for emo_filter in emo_filter_list:
    if (emo_filter == 'gpt'):
        labelPredictionsGPT('gpt', 'emo')
    else: 
        labelPredictionsGPT(emo_filter, emo_filter)

In [ ]:
import pandas as pd
import evaluate

from pprint import pprint
from statistics import mean

# Exact Match scores emotion transition
def exact(truths, preds):
    exact = evaluate.load('exact_match')
    result = exact.compute(predictions = preds, references = truths)['exact_match']
    return result

# BLEU, Google_BLEU, ROUGE, and METEOR score paraphrasing
def bleu(truths, preds):
    bleu = evaluate.load('bleu')
    result = bleu.compute(predictions = preds, references = truths)['bleu']
    return result

def google_bleu(truths, preds):
    google_bleu = evaluate.load('google_bleu')
    result = google_bleu.compute(predictions = preds, references = truths)['google_bleu']
    return result

def rouge1(truths, preds):
    rouge = evaluate.load('rouge')
    result = rouge.compute(predictions = preds, references = truths)['rouge1']
    return result
    
def rouge2(truths, preds):
    rouge = evaluate.load('rouge')
    result = rouge.compute(predictions = preds, references = truths)['rouge2']
    return result
    
def rougeL(truths, preds):
    rouge = evaluate.load('rouge')
    result = rouge.compute(predictions = preds, references = truths)['rougeL']
    return result

def bertscore(truths, preds):
    bscore = evaluate.load('bertscore')
    result = bscore.compute(predictions = preds, references = truths, model_type="distilbert-base-uncased")
    return result

def meteor(truths, preds):
    meteor = evaluate.load('meteor')
    result = meteor.compute(predictions = preds, references = truths)['meteor']
    return result


def scoreSidGPT(model_emo_filter, eval_emo_filter):
    model_name = f"{training_type}-{model_dataset}-{model_emo_filter}-{epochs}epochs-gpt"
    print_base_info(model_name, testing_set, eval_emo_filter)
    df = pd.read_csv(f'predictions-data/{model_name}/{testing_set}-{eval_emo_filter}-prediction_emo.tsv', sep='\t').astype(str)
    
    row_count = 0
    lowered_intensity = 0
    
    for index, row in df.iterrows():
        row_count += 1
        if float(row.input_emo) > float(row.prediction_emo):
            lowered_intensity += 1
    
    pprint("Exact Score")
    pprint(lowered_intensity/row_count)

    pprint("BLEU Score")
    pprint(bleu(df["target_text"], df["predictions"]))

    pprint('Google BLEU Score')
    pprint(google_bleu(df["target_text"], df["predictions"]))

    pprint('ROUGE1 Score')
    pprint(rouge1(df["target_text"], df["predictions"]))

    pprint('ROUGE2 Score')
    pprint(rouge2(df["target_text"], df["predictions"]))

    pprint('ROUGEL Score')
    pprint(rougeL(df["target_text"], df["predictions"]))

    pprint('METEOR Score')
    pprint(meteor(df["target_text"], df["predictions"]))

    print('\n')

def scorePredictionsGPT(model_emo_filter, eval_emo_filter):
    model_name = f"{training_type}-{model_dataset}-{model_emo_filter}-{epochs}epochs-gpt"
    print_base_info(model_name, testing_set, eval_emo_filter)
    df = pd.read_csv(f'predictions-data/{model_name}/{testing_set}-{eval_emo_filter}-prediction_emo.tsv', sep='\t').astype(str)
    
    pprint("Exact Score")
    pprint(exact(df["target_emo"], df["prediction_emo"]))

    pprint("BLEU Score")
    pprint(bleu(df["target_text"], df["predictions"]))

    pprint('Google BLEU Score')
    pprint(google_bleu(df["target_text"], df["predictions"]))

    pprint('ROUGE1 Score')
    pprint(rouge1(df["target_text"], df["predictions"]))

    pprint('ROUGE2 Score')
    pprint(rouge2(df["target_text"], df["predictions"]))

    pprint('ROUGEL Score')
    pprint(rougeL(df["target_text"], df["predictions"]))

    pprint('METEOR Score')
    pprint(meteor(df["target_text"], df["predictions"]))

    print('\n')
    
def orgScoreSidGPT(model_emo_filter, eval_emo_filter):
    model_name = f"{training_type}-{model_dataset}-{model_emo_filter}-{epochs}epochs-gpt"
    print_base_info(model_name, testing_set, eval_emo_filter)
    df = pd.read_csv(f'predictions-data/{model_name}/{testing_set}-{eval_emo_filter}-prediction_emo.tsv', sep='\t').astype(str)
    
    outputs = []
    
    row_count = 0
    lowered_intensity = 0
    
    for index, row in df.iterrows():
        row_count += 1
        if float(row.input_emo) > float(row.prediction_emo):
            lowered_intensity += 1
    
    outputs.append(lowered_intensity/row_count)
    outputs.append(bleu(df["target_text"], df["predictions"]))
    outputs.append(google_bleu(df["target_text"], df["predictions"]))
    outputs.append(rouge1(df["target_text"], df["predictions"]))
    outputs.append(rouge2(df["target_text"], df["predictions"]))
    outputs.append(rougeL(df["target_text"], df["predictions"]))
    outputs.append(meteor(df["target_text"], df["predictions"]))

    for metric in outputs:
        pprint(metric)
    
    print('\n')
    
def orgScorePredictionsGPT(model_emo_filter, eval_emo_filter):
    model_name = f"{training_type}-{model_dataset}-{model_emo_filter}-{epochs}epochs-gpt"
    print_base_info(model_name, testing_set, eval_emo_filter)
    df = pd.read_csv(f'predictions-data/{model_name}/{testing_set}-{eval_emo_filter}-prediction_emo.tsv', sep='\t').astype(str)
    
    outputs = []
    
    outputs.append(exact(df["target_emo"], df["prediction_emo"]))
    outputs.append(bleu(df["target_text"], df["predictions"]))
    outputs.append(google_bleu(df["target_text"], df["predictions"]))
    outputs.append(rouge1(df["target_text"], df["predictions"]))
    outputs.append(rouge2(df["target_text"], df["predictions"]))
    outputs.append(rougeL(df["target_text"], df["predictions"]))
    outputs.append(meteor(df["target_text"], df["predictions"]))

    for metric in outputs:
        pprint(metric)
    
    print('\n')
    
for emo_filter in emo_filter_list:
    if emo_filter == 'sid':
        orgScoreSidGPT(emo_filter, emo_filter)
    else:
        if (emo_filter == 'gpt'):
            orgScorePredictionsGPT('gpt', 'emo')
        else: 
            orgScorePredictionsGPT(emo_filter, emo_filter)